In [1]:
# Core PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# Data loading and augmentation
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms

# Clustering (for offline K-means)
from sklearn.cluster import KMeans
import numpy as np

# Optional utilities
import random, copy, math

c:\Users\tejas_1n\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomHorizontalFlip(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                download=True, transform=transform)

testset = datasets.CIFAR10(root='./data', train=False,
                                download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


### Create a class to make a Semi Supervised dataloader

In [3]:
def generateLabUnlabLoaders(trainset, batch_size_label=64, batch_size_unlabel=1000, M=1000):
    indices = list(range(len(trainset)))
    random.shuffle(indices) 
    labeled_indices = indices[M:]
    unlabeled_indices = indices[:M]
    trainloader = DataLoader(Subset(trainset, labeled_indices), batch_size=batch_size_label,
                                          shuffle=True, num_workers=2)
    if M == 0:
        train_unlabeled_loader = None
    else:
        train_unlabeled_loader = DataLoader(Subset(trainset, unlabeled_indices), batch_size=batch_size_unlabel,
                                          shuffle=True, num_workers=2)
    return trainloader, train_unlabeled_loader


generateLabUnlabLoaders(trainset)

(<torch.utils.data.dataloader.DataLoader at 0x1f565e0ec10>,
 <torch.utils.data.dataloader.DataLoader at 0x1f565e0ee20>)


### Create a basic CNN model for dimensionality reduction

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x, return_logits=False):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x_logits = F.relu(self.fc2(x))
        x = self.fc3(x_logits)
        if return_logits:
            return x, x_logits
        else:
            return x

### Train on dataset

In [5]:
model = SimpleCNN()
optim = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [6]:
strong_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=32, padding=4),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2),
])

weak_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=32, padding=4),
])


In [7]:
print(len(testset))

10000


In [ ]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'
if device == 'gpu':
    model = model.cuda()
    criterion = criterion.cuda()

# indices = list(range(len(trainset)))
# random.shuffle(indices)
# trainset_reduced = Subset(trainset, indices[:TRAIN_REDUCE_SIZE])
trainloader_labeled, trainloader_unlabeled =  generateLabUnlabLoaders(trainset, 64, 512, 20000)
testloader = DataLoader(testset, batch_size=64, num_workers=2)

labeled_losses = []
unlabeled_losses = []
valid_loss = []

for epoch in range(20): 
    for i, (labeled_x, labeled_y) in enumerate(tqdm(trainloader_labeled)):
        labeled_x = labeled_x.to(device)
        labeled_y = labeled_y.to(device)

        z_labeled, logits_labeled = model(labeled_x, return_logits=True)

        L_sup = criterion(z_labeled, labeled_y)

        optim.zero_grad()
        L_sup.backward()
        optim.step()


    L_kmeans = 0
    L_consistency = 0
    if trainloader_unlabeled:
        for i, data in enumerate(tqdm(trainloader_unlabeled)):
            unlabeled_x_batch, _ = data # ignore labels
            z_unlabeled, logits_unlabeled = model(unlabeled_x_batch, return_logits=True)
            with torch.no_grad():
                z_unlabeled_np = z_unlabeled.cpu().numpy()
                kmeans = KMeans(n_clusters=10, n_init=10).fit(z_unlabeled_np)
                pseudo_labels = torch.tensor(kmeans.labels_).to(torch.long).to(device)
                cluster_centers = torch.tensor(kmeans.cluster_centers_).to(device)
            L_kmeans = torch.mean((z_unlabeled - cluster_centers[pseudo_labels]) ** 2) 

            # ----- Loss 3: Consistency -----
            x_weak, x_strong = weak_transform(unlabeled_x_batch), strong_transform(unlabeled_x_batch)
            z_weak, logits_weak = model(x_weak, return_logits=True)
            z_strong, logits_strong = model(x_strong, return_logits=True)
            L_consistency = F.kl_div(F.log_softmax(logits_weak, dim=1), F.softmax(logits_strong, dim=1), reduction='batchmean')

            loss_overall = L_kmeans * 0.25 + L_consistency * 0.25

            optim.zero_grad()
            loss_overall.backward()
            optim.step()
    
    with torch.no_grad():
        total_correct = 0
        total_loss = 0

        for i, (test_x, test_y) in enumerate(testloader):
            test_x.to(device)
            test_y.to(device)
            z_test = model(test_x, return_logits=False)
            z_cats = torch.argmax(z_test, dim=1)
            total_correct += torch.sum(z_cats == test_y).item()
            total_loss += criterion(z_test, z_cats)
        
        print(f'Epoch: {epoch+1}/10, Correct: {total_correct/10000}, Loss: {total_loss/10000}')



  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:47<1:18:42, 47.70s/it]

Epoch: 1/10, Correct: 0.6335, Loss: 0.008509279228746891


  2%|▏         | 2/100 [01:47<1:29:10, 54.60s/it]

Epoch: 2/10, Correct: 0.6906, Loss: 0.006907481234520674


...

Epoch: 98/10, Correct: 0.7425, Loss: 0.0007366041536442935


 99%|█████████▉| 99/100 [1:53:10<01:25, 85.28s/it]

Epoch: 99/10, Correct: 0.7489, Loss: 0.0007630593026988208


100%|██████████| 100/100 [1:54:36<00:00, 68.77s/it]

Epoch: 100/10, Correct: 0.7432, Loss: 0.0008022922556847334
